In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# TF IDF SVM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# panggil dataset
df = pd.read_csv('Whatsapp.csv')
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,"['respond', 'message', 'android', 'auto', 'ack...",negative
1,1,"['I', 've', 'use', 'app', 'nearly', 'decade', ...",positive
2,2,"['daily', 'spam', 'text', 'call', 'get', 'app'...",positive
3,3,"['problem', 'perfection', 'make', 'change', 'd...",positive
4,4,"['good', 'interface', 'nice', 'functionality',...",positive


In [ ]:
# Mengubah teks lemmatized ke format yang sesuai
df['text_lemmatized'] = df['text_lemmatized'].apply(lambda x: ' '.join(eval(x)))
df.head()

,Unnamed: 0,text_lemmatized,label
0,0,respond message android auto acknowledge I ve ...,negative
1,1,I ve use app nearly decade though new useful f...,positive
2,2,daily spam text call get app take put two two ...,positive
3,3,problem perfection make change downgrade do no...,positive
4,4,good interface nice functionality need wish wo...,positive


In [ ]:
# mengubah label menjadi format numerik
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# membagi data
X = df['text_lemmatized']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(68514,) (29364,) (68514,) (29364,)


In [ ]:
# ekstraksi fitur dengan TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(X_train_tfidf.shape, X_test_tfidf.shape)

(68514, 36032) (29364, 36032)


In [ ]:
# melatih model SVM dengan kernel linear dan class weight 'balanced'
svm_model = SVC(kernel='linear', class_weight='balanced')
svm_model.fit(X_train_tfidf, y_train)

SVC(class_weight='balanced', kernel='linear')

In [ ]:
# melakukan prediksi dengan data latih
y_train_pred = svm_model.predict(X_train_tfidf)
print("Data Latih:")
print(classification_report(y_train, y_train_pred, target_names=label_encoder.classes_))

Data Latih:
              precision    recall  f1-score   support

    negative       0.91      0.96      0.93     19117
     neutral       0.90      0.98      0.94      9741
    positive       0.99      0.94      0.96     39656

    accuracy                           0.95     68514
   macro avg       0.93      0.96      0.95     68514
weighted avg       0.95      0.95      0.95     68514



In [ ]:
# melakukan prediksi dengan data uji
y_test_pred = svm_model.predict(X_test_tfidf)
print("Data Uji:")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))

Data Uji:
              precision    recall  f1-score   support

    negative       0.87      0.93      0.90      8157
     neutral       0.86      0.96      0.91      4100
    positive       0.98      0.92      0.94     17107

    accuracy                           0.93     29364
   macro avg       0.90      0.94      0.92     29364
weighted avg       0.93      0.93      0.93     29364



In [ ]:
def predict_user_input(text, vectorizer, model, label_encoder):
    # ekstraksi fitur dengan TF-IDF
    text_tfidf = vectorizer.transform([text])

    # model melakukan prediksi
    prediction = model.predict(text_tfidf)

    # mengubah kelas prediksi menjadi label asli
    predicted_label = label_encoder.inverse_transform(prediction)
    return predicted_label[0]

# mencoba prediksi dengan ketikan dari user
user_input = input("Input Teks: ")
predicted_sentiment = predict_user_input(user_input, vectorizer, svm_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: great
teks ini masuk ke : positive


In [ ]:
# mencoba prediksi dengan ketikan dari user
user_input = input("Input Teks: ")
predicted_sentiment = predict_user_input(user_input, vectorizer, svm_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: bad
teks ini masuk ke : negative


In [ ]:
# mencoba prediksi dengan ketikan dari user
user_input = input("Input Teks: ")
predicted_sentiment = predict_user_input(user_input, vectorizer, svm_model, label_encoder)
print(f"teks ini masuk ke : {predicted_sentiment}")

Input Teks: standart
teks ini masuk ke : neutral
